<a href="https://colab.research.google.com/github/maham-patel/CI--EA-implementation/blob/main/EA%20class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# generate random integer values
from random import seed
from random import randint
from random import random
import math
import copy

class EA_imp():  
  def __init__(self, genes, population_size, offspring_size, generations, mutation_rate, iterations, parent_ss, survivor_ss, max_min):
    self.population_size = population_size
    self.offspring_size = offspring_size
    self.generations = generations
    self.mutation_rate = mutation_rate
    self.iterations = iterations
    self.BSF_table = [[0]*(self.iterations+1)]*self.generations
    self.ASF_table = [[0]*(self.iterations+1)]*self.generations
    self.population = []
    self.parent_ss = parent_ss
    self.survivor_ss = survivor_ss
    self.gen_num = 0
    self.iter_num = 0
    self.max_min = max_min
    #missing parent and child ss selection

    def generate_gen1(self):
      self.population = []
      for i in range(self.population_size):
        individual = self.generate_individual() #individual = ([list of gene numbers], fitness)
        fitness = self.fitness_function(individual) #calculate fitness of individual
        self.population.append(tuple(individual, fitness))
    
    def check_repetition(self, chromosome, gene):
      for i in chromosome:
        if i == gene:
          return True
      return False

    def generate_individual(self):
      seed()
      size = len(self.genes)
      individual = []
      i = 1
      while i != size:
        gene = randint(1, size)
        if self.check_repetition(individual, gene) == False:
          individual.append(gene)
          i += 1
      return individual

    def fitness_function(self, chromosome):
      print("THIS SHOULD BE INHERITED")

    def parent_selection(self):
      for i in range(self.offspring_size/2):
        if self.parent_ss == "FPS":
          parent_1 = self.FPS()
          parent_2 = self.FPS()
        elif self.parent_ss == "RBS":
          parent_1 = self.RBS()
          parent_2 = self.RBS()
        elif self.parent_ss == "BT":
          parent_1 = self.BT()
          parent_2 = self.BT()
        elif self.parent_ss == "Truncation":
          parent_1 = self.Truncation()
          parent_2 = self.Truncation()
        else:
          parent_1 = self.Random()
          parent_2 = self.Random()
        offspring1, offspring2 = self.crossover_operator_order2(parent_1[0], parent_2[0])
        offspring1 = self.mutation_operator(offspring1)
        offspring2 = self.mutation_operator(offspring2)
        fitness1 = self.fitness_function(offspring1)
        fitness2 = self.fitness_function(offspring2)
        self.population.append(tuple(offspring1, fitness1))
        self.population.append(tuple(offspring2, fitness2))
    
    def survivor_selection(self):
      new_population = []
      for i in range(self.population_size):
        if self.survivor_ss == "FPS":
          survivor = self.FPS()
        elif self.survivor_ss == "RBS":
          survivor = self.RBS()
        elif self.survivor_ss == "BT":
          survivor = self.BT()
        elif self.survivor_ss == "Truncation":
          survivor = self.Truncation()
        else:
          survivor = self.Random()
        new_population.append(survivor)
      self.population = new_population

    def crossover_operator_order2(self, parent1, parent2):
      seed()
      size = len(self.genes)
      cutoff1 = randint(0, size-1)
      cutoff2 = randint(0, size-1)
      while abs(cutoff1 - cutoff2) < int(0.1*size):
        cutoff2 = randint(0, size-1)
      l = -1
      h = -1
      if cutoff1 > cutoff2:
        h = cutoff1
        l = cutoff2
      else:
        h = cutoff2
        l = cutoff1
      print('h = ', h)
      print('l = ', l )
      offspring1 = [-1]*size
      offspring2 = [-1]*size
      offspring1[l:h] = parent1[l:h]
      offspring2[l:h] = parent2[l:h]
      n = abs(cutoff1 - cutoff2)
      i1 = copy.deepcopy(h)
      i2 = copy.deepcopy(h)
      j1 = copy.deepcopy(h)
      j2 = copy.deepcopy(h)
      while n != size:
        while check_repetition(offspring1, parent2[i2]):
          i2 += 1
          if i2 == size:
            i2 = 0
        while check_repetition(offspring2, parent1[j2]):
          j2 += 1
          if j2 == size:
            j2 = 0
        offspring1[i1] = parent2[i2]
        offspring2[j1] = parent1[j2]
        i1 += 1
        i2 += 1
        j1 += 1
        j2 += 1
        if i1 == size:
          i1 = 0
        if j1 == size:
          j1 = 0
        if i2 == size:
          i2 = 0
        if j2 == size:
          j2 = 0
        n += 1
      return offspring1, offspring2

    def crossover_operator_order1(self, parent_1, parent_2):
      pass

    def mutation_operator(self, individual):
      seed()
      rand_num = randint(0, 100)
      if rand_num <= self.mutation_rate*100:
        pos1 = randint(0, len(individual)-1)
        pos2 = randint(0, len(individual)-1)
        individual[pos1], individual[pos2] = individual[pos2], individual[pos1]
      return individual

    def end_of_gen(self):
      avg = 0
      if self.max_min == 'max':
        BSF = -math.inf
        for individual, fitness in self.population:
          if fitness > BSF:
            BSF = fitness
          avg += fitness
      else:
        BSF = math.inf
        for individual, fitness in self.population:
          if fitness < BSF:
            BSF = fitness
          avg += fitness
      ASF = avg/len(self.genes)
      self.BSF_table[self.gen_num][self.iter_num] = BSF
      self.ASF_table[self.gen_num][self.iter_num] = ASF
      self.gen_num += 1

    def end_of_iter(self):
      for gen in range(self.generations):
        sum_BSF = sum(BSF_table[gen])- BSF_table[gen][-1]
        sum_ASF = sum(ASF_table[gen])- ASF_table[gen][-1]
        BSF_table[gen][-1] = sum_BSF/(iter_num + 1)
        ASF_table[gen][-1] = sum_ASF/(iter_num + 1)
      self.iter_num += 1

    def insertionSort(self, population):
      lst = []
      total_rank = 0
      cur = 0
      for i in population:
        _, rank = i
        j = 0
        inserted = False
        while j < len(lst):
          if rank < lst[j][1]:
            lst.insert(j, i)
            inserted = True
            break
          else:
            j += 1
        if inserted == False:
          lst.append(i)
        cur += 1
        total_rank += cur
      return lst, total_rank

    def FPS(self):
      seed()
      rand_num = rand()
      total_fitness = 0
      for individual, fitness in self.population:
        total_fitness += fitness 
      current_cumulation = 0
      for chromosome in self.population:
        individual, fitness = chromosome
        if self.max_min == 'max':
          current_cumulation += fitness/total_fitness
        else:
          current_cumulation += (1- fitness/total_fitness)/(self.population_size -1)
        if rand_num <= current_cumulation:
          return chromosome

    def RBS(self):
      seed()
      rand_num = rand()
      ranked_arrangement, total_rank = self.insertionSort(self.population)
      if self.max_min == 'min':
        ranked_arrangement = ranked_arrangement[::-1]
      current_cumulation = 0
      for i in range(len(ranked_arrangement)):
        chromosome = ranked_arrangement[i]
        current_cumulation += (i+1)/total_rank
        if rand_num <= current_cumulation:
          return chromosome

    def BT(self):
      seed()
      player1 = self.population[randint(0, (self.population_size-1))]
      player2 = self.population[randint(0, (self.population_size-1))]
      if self.max_min == 'max':
        if player1[1] > player2[1]:
          return player1
        else:
          return player2
      else:
        if player1[1] > player2[1]:
          return player2
        else:
          return player1

    def Truncation(self):
      winner = -1
      if self.max_min == "max":
        best_fitness = -math.inf
      else:
        best_fitness = math.inf
      for i in range(len(self.population)):
        individual = population[i]
        if self.max_min == 'max' and individual[1] > best_fitness:
          best_fitness = individual[1]
          winner = individual
        if self.max_min == 'min' and individual[1] < best_fitness:
          best_fitness = individual[1]
          winner = individual
      return winner
          
    def Random(self):
      seed()
      size = len(self.genes)
      rand_num = randint(0, size-1)
      return population[rand_num]

    def run_algo(self):
      self.iter_num = 0
      while self.iter_num < self.iterations:
        self.generate_gen1()
        self.gen_num = 0
        while self.gen_num < self.generations:
          self.parent_selection()
          self.survivor_selection()
          self.end_of_gen()
        self.end_of_iter()
      self.show_result()

    def show_result(self):
      print("BSF TABLE")
      for gen_num in range(len(self.BSF_table)):
        print(gen_num+1, ": ", gen)
      print("\n\n")
      print("*  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *  *")
      print("\n\n")
      print("BSF TABLE")
      for gen_num in range(len(self.ASF_table)):
        print(gen_num+1, ": ", gen)

      


In [ ]:
import matplotlib.pyplot as plt
import tsplib95
class TSP(EA_imp):
  def __init__(self, file_, population_size, offspring_size, generations, mutation_rate, iterations, parent_ss, survivor_ss):
    with open('file_') as f:
      text = f.read()
      problem = tsplib95.parse(text)
    genes = problem.node_coords
    max_min = 'min'
    self.distances_tab = [[0]*len(genes)]*len(genes)
    EA_imp.__init__(self, genes, population_size, offspring_size, generations, mutation_rate, iterations, parent_ss, survivor_ss, max_min)
    
    for i in range(len(genes)):
      for j in range(len(genes)):
        c1 = genes[i+1]
        c2 = genes[j+1]
        distance = math.sqrt((c1[0]-c2[0])**2 + (c1[1]-c2[1])**2)
        self.distances_tab[i][j] = distance

  def fitness_function(self, chromosome):
    total_dist = 0
    for gene_num in range(1, len(chromosome)):
      gene1 = chromosome[gene_num]
      gene2 = chromosome[gene_num-1]
      total_dist += self.distances_tab[gene1][gene2]
    return total_dist

    


